## Condo Prices Prediction in Toronto

### The Toronto condo market has been heated in the past few years with its economy recovering, and open immigration policies leading to overall population growth. 

### This project aims to predict listing prices of condo units in Toronto based on their individual characteristics. Different features were derived from raw data, and various models were employed in order to attain best accuracy. 

![Housing_Image](./TOhousingimage.jpeg)

#### Data were previously collected. Listings were mainly from 2018.

### EDA

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [2]:
import warnings
warnings.filterwarnings('ignore')
# Override some default panda options to make it easy to display data
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

In [3]:
# Read csv data into a pandas dataframe
column_name = ['id', 'link', 'name', 'size_range','listing_price', 'price_per_ft2', 'bed', 'shower', 'parking', 'dom', 'maint_fee', 'region', 'street', 'unit_num', 'latitude','longitude', 'locker', 'taxes', 'exposure', 'balcony', 'description','pics','floorplans']

df = pd.read_csv('aug8_soups.csv', delimiter = '|',header = None, names = column_name)

In [4]:
df.head(5)

,id,link,name,size_range,listing_price,price_per_ft2,bed,shower,parking,dom,maint_fee,region,street,unit_num,latitude,longitude,locker,taxes,exposure,balcony,description,pics,floorplans
0,C4213966,https://condos.ca/toronto/studio-2-30-nelson-s...,"Studio 2, #1004",700-799,"$798,000","$1,047",2,2,Yes,0,$480,CA-ON,30 Nelson St,1004,43.649100,-79.38840,No,"$3,109",Se,Open,"Quality Built Studio 2 Condo By Aspen Ridge,Br...",['https://condos.ca/public/condo_listing/91/4b...,NaN
1,E4213970,https://condos.ca/toronto/guildwood-terrace-32...,"Guildwood Terrace, #802",1000-1199,"$399,000",$337,2+1,2,Yes,0,$920,CA-ON,3231-3233 Eglinton Ave E,802,43.744281,-79.21371,Exclusive,"$1,394",E,None,Spacious & Bright 1200 Sqft. Corner Unit With ...,['https://condos.ca/public/condo_listing/fa/4c...,NaN
2,W4213969,https://condos.ca/toronto/the-2800-condos-2800...,The 2800 Condos,900-999,"$573,800",$N/A,2+1,2,No,0,$0,CA-ON,2800 Keele St,807,43.728700,-79.48280,No,Unknown,Nw,Open,Please Log in to seeBroker: DANMAR EMPIRE REAL...,['https://condos.ca//public/condo_building/f0/...,NaN
3,W4213978,https://condos.ca/toronto/palace-place-1-palac...,"Palace Place, #4011",1000-1199,"$589,000",$N/A,1,2,Yes,0,$996,CA-ON,1 Palace Pier Crt,4011,43.631400,-79.47250,Owned,"$2,686",Ne,None,Palace Place! Largest 1 Bedroom Suite In The B...,['https://condos.ca/public/condo_listing/1e/4e...,NaN
4,C4212803,https://condos.ca/toronto/the-mansions-on-jarv...,The Mansions on Jarvis,500-599,"$439,000",$784,1,1,No,0,$217,CA-ON,539 Jarvis St,GT3,43.667600,-79.37880,Owned,"$2,119",S,Juliette,Please Log in to seeBroker: ROYAL LEPAGE REAL ...,['https://condos.ca//public/condo_building/0c/...,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 23 columns):
id               2159 non-null object
link             2159 non-null object
name             2159 non-null object
size_range       2159 non-null object
listing_price    2159 non-null object
price_per_ft2    1926 non-null object
bed              2157 non-null object
shower           2159 non-null int64
parking          2159 non-null object
dom              2159 non-null int64
maint_fee        2159 non-null object
region           2159 non-null object
street           2107 non-null object
unit_num         2156 non-null object
latitude         2159 non-null float64
longitude        2159 non-null float64
locker           2159 non-null object
taxes            2159 non-null object
exposure         2157 non-null object
balcony          2159 non-null object
description      2159 non-null object
pics             2159 non-null object
floorplans       1 non-null object
dtypes: float64(2), in

In [6]:
df.describe()

,shower,dom,latitude,longitude
count,2159.000000,2159.000000,2159.000000,2159.000000
mean,1.616489,49.167670,43.707152,-79.397517
std,0.673355,100.772841,0.062603,0.079238
min,1.000000,0.000000,43.587655,-79.615900
25%,1.000000,7.000000,43.645500,-79.430300
50%,2.000000,31.000000,43.701300,-79.395200
75%,2.000000,62.000000,43.769813,-79.365900
max,16.000000,961.000000,43.826300,-79.169400


#### As we can see, there are a total of 23 columns, and 4 of them contains numerical data, the rest are either categorical or need to be transformed. 

#### First, we split the name column as it contains crucial information of the unit number, from which we can derive the floor number

In [7]:
df_split_name = df['name'].str.split(',', expand= True).rename(columns={0:'Condo_name', 1:'suite_num'})
df_split_name = df_split_name.drop([2,3], 1)

In [8]:
df_concat = pd.concat([df, df_split_name], axis= 1)
# df_concat['suite_num_clean'] = [room.strip().strip('#') if room else room for room in df_concat.suite_num]

In [9]:
# df_concat

In [10]:
import re


def process_room_num(df):
    
    floor_ls = []
    unit_ls = []
    for room in df['unit_num']:
        floor = 9999
        unit = 9999
#         print(room)

        if room and room != ' ' and not str(room) == 'nan':

            clean_room = room.strip('#').strip()


            if clean_room.isdigit():
                if len(clean_room) ==1:
                    floor = 1
                    unit = clean_room
                    floor_ls.append(floor)
                    unit_ls.append(unit) 

                elif len(clean_room) ==2:
                    floor = 1
                    unit = clean_room
                    floor_ls.append(floor)
                    unit_ls.append(unit) 

                elif len(clean_room) ==3:
                    floor = clean_room[0]
                    unit = clean_room[1:]
                    floor_ls.append(floor)
                    unit_ls.append(unit)

                elif len(clean_room) ==4:
                    floor = clean_room[:2]
                    unit = clean_room[2:]
                    floor_ls.append(floor)
                    unit_ls.append(unit)

                else:
                    print('check digits!')
                    floor_ls.append(floor)
                    unit_ls.append(unit)

            elif not clean_room.isdigit() and re.match('(?i)[a-z]\d{2}', clean_room.lower()):
                floor = 1
                unit = clean_room[1:]
                floor_ls.append(floor)
                unit_ls.append(unit)

            elif not clean_room.isdigit() and re.match('(?i)[a-z]\d{2}', clean_room.lower()):
                floor = clean_room[1]
                unit = clean_room[2:]
                floor_ls.append(floor)
                unit_ls.append(unit)

            elif not clean_room.isdigit() and re.match('\d{4}\w$', clean_room.lower()):
                floor = clean_room[:2]
                unit = clean_room[2:4]
                floor_ls.append(floor)
                unit_ls.append(unit)

            elif not clean_room.isdigit() and re.match('\d{3}\w$', clean_room.lower()):
                floor = clean_room[0]
                unit = clean_room[1:3]
                floor_ls.append(floor)
                unit_ls.append(unit)

            elif not clean_room.isdigit() and 'th' in clean_room.lower():
                floor = 1
                unit = clean_room.lower().strip('th').strip('-').strip()
                floor_ls.append(floor)
                unit_ls.append(unit)

            elif not clean_room.isdigit() and 'gt' in clean_room.lower():
                floor = 1
                unit = clean_room.lower().strip('gt').strip('-').strip()
                floor_ls.append(floor)
                unit_ls.append(unit)            

            elif not clean_room.isdigit() and 'uph' in clean_room.lower(): #fix this check uph3; maybe not, top floor anyways
                floor = 'UPH'
                unit = clean_room.lower().strip('uph').strip()
                floor_ls.append(floor)
                unit_ls.append(unit)

            elif not clean_room.isdigit() and 'ph' in clean_room.lower():
                floor = 'PH'
                unit = clean_room.lower().strip('ph').strip()
                floor_ls.append(floor)
                unit_ls.append(unit)

            elif not clean_room.isdigit() and ('lph' in clean_room.lower() or 'lp' in clean_room.lower()):
                floor = 'LPH' 
                unit = clean_room.lower().strip('lph').strip()
                floor_ls.append(floor)
                unit_ls.append(unit)

            else:
                floor_ls = floor_ls
                unit_ls = unit_ls
                floor_ls.append(floor)
                unit_ls.append(unit)

            # print('IF Floor: {}, unit: {}'.format(floor, unit))
            

        else:

            floor = 0
            unit = '0000'
            floor_ls.append(floor)
            unit_ls.append(unit)

            # print('ELSE Floor: {}, unit: {}'.format(floor, unit))
    return floor_ls, unit_ls

In [11]:
df_concat['floor'], df_concat['room_num']  = process_room_num(df_concat)

#### Now we read in assciated condo building information. 

In [12]:
building_column_names = ['Condo_name', 'Building_url', 'Address', 'Pre_constr', 'Neighbourhood', 'Corp#', 'Expected_Completion', 'Developer', 'Number_Units', 'Storeys',
                            'Property_Mgmt_Co', 'Smallest_Suite', 'Largest_Suite', 'Transit_score', 'Description', 'Maint_fee_covers']
df_building = pd.read_csv('20180917_buildings.csv', delimiter = '|',header = None, names = building_column_names)

In [13]:
df_building.head(3)

,Condo_name,Building_url,Address,Pre_constr,Neighbourhood,Corp#,Expected_Completion,Developer,Number_Units,Storeys,Property_Mgmt_Co,Smallest_Suite,Largest_Suite,Transit_score,Description,Maint_fee_covers
0,Wynford Green,https://condos.ca/new-development/wynford-gree...,844 Don Mills Road,Pre-Construction,Banbury | Don Mills,TBD,2020,Context Development,3887,44,Unknown,400.0,1700.0,77,These high-rise Toronto condos are expected to...,"['Common Element Maintenance', 'Building Insur..."
1,The Prestige at Pinnacle Condos,https://condos.ca/new-development/the-prestige...,1 Yonge Street,Pre-Construction,The Waterfront,TBD,2020,Pinnacle International,3885,95,Unknown,600.0,1300.0,100,These high-rise Toronto condos are expected to...,"['Common Element Maintenance', 'Building Insur..."
2,Galleria Mall Condos,https://condos.ca/new-development/galleria-mal...,1245 Dupont Street,Pre-Construction,Dovercourt-Wallace Emerson-Junction,TBD,2020,Freed Developments,3400,47,Unknown,400.0,1200.0,83,Located in the Dovercourt-Wallace Emerson-Junc...,"['Common Element Maintenance', 'Building Insur..."


In [14]:
df_join = pd.merge(df_concat, df_building, how= 'left', on = 'Condo_name')

In [15]:
df_join.head(3)

,id,link,name,size_range,listing_price,price_per_ft2,bed,shower,parking,dom,maint_fee,region,street,unit_num,latitude,longitude,locker,taxes,exposure,balcony,description,pics,floorplans,Condo_name,suite_num,floor,room_num,Building_url,Address,Pre_constr,Neighbourhood,Corp#,Expected_Completion,Developer,Number_Units,Storeys,Property_Mgmt_Co,Smallest_Suite,Largest_Suite,Transit_score,Description,Maint_fee_covers
0,C4213966,https://condos.ca/toronto/studio-2-30-nelson-s...,"Studio 2, #1004",700-799,"$798,000","$1,047",2,2,Yes,0,$480,CA-ON,30 Nelson St,1004,43.649100,-79.38840,No,"$3,109",Se,Open,"Quality Built Studio 2 Condo By Aspen Ridge,Br...",['https://condos.ca/public/condo_listing/91/4b...,NaN,Studio 2,#1004,10,04,https://condos.ca/toronto/studio-2-30-nelson-st,30 Nelson St,NaN,Queen West,TSCC 2573,NaN,Aspen Ridge Homes,411.0,41.0,Icc Property Management,413.0,2573.0,100,From the Builder Studio on Richmond is the qui...,"['Gym / Exercise Room', 'Common Rooftop Deck',..."
1,E4213970,https://condos.ca/toronto/guildwood-terrace-32...,"Guildwood Terrace, #802",1000-1199,"$399,000",$337,2+1,2,Yes,0,$920,CA-ON,3231-3233 Eglinton Ave E,802,43.744281,-79.21371,Exclusive,"$1,394",E,None,Spacious & Bright 1200 Sqft. Corner Unit With ...,['https://condos.ca/public/condo_listing/fa/4c...,NaN,Guildwood Terrace,#802,8,02,https://condos.ca/toronto/guildwood-terrace-32...,3231-3233 Eglinton Ave E,NaN,Scarborough Village,MTCC 1013,NaN,Tridel,349.0,20.0,Gpm Property Management,690.0,1463.0,67,Located at 3231-3233 Eglinton Avenue East in S...,"['Gym / Exercise Room', 'Pool', 'Parking Garag..."
2,W4213969,https://condos.ca/toronto/the-2800-condos-2800...,The 2800 Condos,900-999,"$573,800",$N/A,2+1,2,No,0,$0,CA-ON,2800 Keele St,807,43.728700,-79.48280,No,Unknown,Nw,Open,Please Log in to seeBroker: DANMAR EMPIRE REAL...,['https://condos.ca//public/condo_building/f0/...,NaN,The 2800 Condos,None,8,07,https://condos.ca/toronto/the-2800-condos-2800...,2800 Keele St,NaN,Downsview | Roding | CFB,TSCC 2637,NaN,QuadCam Development Group,214.0,11.0,Unknown,309.0,1446.0,69,Located in the Downsview-Roding-CFB neighbourh...,"['Gym / Exercise Room', 'Common Rooftop Deck',..."


### Encode categorical data

In [16]:
def encode_cat(df, cat_feature_list):
    for feature in cat_feature_list:
        if feature == 'exposure':
            df[feature] = df[feature].str.lower()
        df = pd.concat([df,pd.get_dummies(df[feature], prefix=feature)],axis=1)
        df.drop([feature],axis=1, inplace=True)
    return df

In [17]:
cat_feature_list = ['balcony', 'bed', 'locker', 'exposure', 'Neighbourhood',
                   'Developer', 'Condo_name', 'Property_Mgmt_Co']
encoded_df = encode_cat(df_join, cat_feature_list)

In [18]:
# For parking, user binarizer to encode as it should be a boolean value
binarizer_prk = preprocessing.LabelBinarizer(sparse_output=False)
encoded_df['parking'] = binarizer_prk.fit_transform(encoded_df['parking'])

In [19]:
encoded_df.head(3)

,id,link,name,size_range,listing_price,price_per_ft2,shower,parking,dom,maint_fee,region,street,unit_num,latitude,longitude,taxes,description,pics,floorplans,suite_num,floor,room_num,Building_url,Address,Pre_constr,Corp#,Expected_Completion,Number_Units,Storeys,Smallest_Suite,Largest_Suite,Transit_score,Description,Maint_fee_covers,balcony_Enclosed,balcony_Juliette,balcony_None,balcony_Open,balcony_Terrace,balcony_Unknown,bed_1,bed_1+1,bed_2,bed_2+1,bed_3,bed_3+1,bed_4,bed_4+1,bed_Studio,bed_Studio & 1,locker_Common,locker_Ensuite,locker_Ensuite-Owned,locker_Exclusive,locker_No,locker_Owned,exposure_e,exposure_ew,exposure_n,exposure_ne,exposure_ns,exposure_nw,exposure_s,exposure_se,exposure_sw,exposure_w,Neighbourhood_Agincourt North,Neighbourhood_Agincourt South-Malvern West,Neighbourhood_Alderwood,Neighbourhood_Annex | U of T,Neighbourhood_Banbury | Don Mills,Neighbourhood_Bathurst Manor,Neighbourhood_Bay St. Corridor,Neighbourhood_Bayview Village,Neighbourhood_Bayview Woods-Steeles,Neighbourhood_Bedford Park-Nortown,Neighbourhood_Beechborough-Greenbrook,Neighbourhood_Bendale,Neighbourhood_Birchcliffe-Cliffside,Neighbourhood_Black Creek,Neighbourhood_Briar Hill-Belgravia,Neighbourhood_Bridle Path | Sunnybrook | York Mills,Neighbourhood_Broadview North,Neighbourhood_Brookhaven-Amesbury,Neighbourhood_Cabbagetown,Neighbourhood_Casa Loma,Neighbourhood_Church St. Corridor,Neighbourhood_CityPlace,Neighbourhood_Clairlea | Birchmount,Neighbourhood_Clanton Park,Neighbourhood_Cliffcrest,Neighbourhood_Corktown,Neighbourhood_Distillery District,Neighbourhood_Don Valley Village,Neighbourhood_Dorset Park,Neighbourhood_Dovercourt-Wallace Emerson-Junction,Neighbourhood_Downsview | Roding | CFB,Neighbourhood_East End-Danforth,Neighbourhood_Edenbridge | Humber Valley,Neighbourhood_Eglinton East,Neighbourhood_Englemount | Lawrence,Neighbourhood_Eringate | Centennial | West Deane,Neighbourhood_Etobicoke West Mall,Neighbourhood_Flemingdon Park,Neighbourhood_Forest Hill North,Neighbourhood_Forest Hill South,Neighbourhood_Fort York,Neighbourhood_Grange Park,Neighbourhood_Guildwood,Neighbourhood_Henry Farm,Neighbourhood_High Park North,Neighbourhood_High Park-Swansea,Neighbourhood_Hillcrest Village,Neighbourhood_Humber Heights | Westmount,Neighbourhood_Humber Summit,Neighbourhood_Humbermede,Neighbourhood_Humewood-Cedarvale,Neighbourhood_Islington | City Centre West,Neighbourhood_Junction Area,Neighbourhood_Keelesdale-Eglinton West,Neighbourhood_Kennedy Park,Neighbourhood_Kensington Market,Neighbourhood_King West,Neighbourhood_Kingsview Village | The Westway,Neighbourhood_L'Amoreaux,Neighbourhood_Lansing-Westgate,Neighbourhood_Lawrence Park North,Neighbourhood_Lawrence Park South,Neighbourhood_Leaside,Neighbourhood_Liberty Village,Neighbourhood_Little Italy,Neighbourhood_Little Portugal,Neighbourhood_Long Branch,Neighbourhood_Malvern,Neighbourhood_Markland Wood,Neighbourhood_Milliken,Neighbourhood_Mimico,Neighbourhood_Morningside,Neighbourhood_Moss Park,Neighbourhood_Mount Dennis,Neighbourhood_Mount Olive | Silverstone | Jamestown,Neighbourhood_Mount Pleasant East,Neighbourhood_Mount Pleasant West,Neighbourhood_Newtonbrook East,Neighbourhood_Newtonbrook West,Neighbourhood_O'Connor-Parkview,Neighbourhood_Oakridge,Neighbourhood_Parkwoods-Donalda,Neighbourhood_Pleasant View,Neighbourhood_Queen West,Neighbourhood_Regent Park,Neighbourhood_Rockcliffe-Smythe,Neighbourhood_Roncesvalles,Neighbourhood_Rosedale-Moore Park,Neighbourhood_Rouge E11,Neighbourhood_Scarborough Village,Neighbourhood_South Parkdale,Neighbourhood_South Riverdale | Leslieville,Neighbourhood_St. Andrew | Windfields,Neighbourhood_St. James Town,Neighbourhood_St. Lawrence,Neighbourhood_Steeles,Neighbourhood_Stonegate-Queensway,Neighbourhood_Tam O'Shanter | Sullivan,Neighbourhood_The Beaches,Neighbourhood_The Core,Neighbourhood_The Waterfront,Neighbourhood_Thorncliffe Park,Neighbourhood_Victoria Village,Neighbourhood_West Hill,Neighbourhood_West Humber | Clairville,Neighbourhood_Westminste

In [20]:
# convert main_fee col to int
encoded_df['maint_fee'] = [int(x.strip('$').replace(",", "")) if type(x)==str else x for x in encoded_df['maint_fee']]

In [21]:
# convert tax col to int
encoded_df['taxes'] = tax_arry = [int(x.strip('$').replace(",", "")) if type(x)==str and x != 'Unknown' else np.NaN for x in encoded_df['taxes']]

### Deal with missing values

In [22]:
encoded_df['taxes'] = encoded_df['taxes'].fillna(value = encoded_df['taxes'].mean())

### Deal with floor numbers

In [23]:
encoded_df['floor_ratio'] = encoded_df[~encoded_df['floor'].isin(['UPH', 'LPH','PH', 9999])]['floor'].astype('int') / \
encoded_df[~encoded_df['floor'].isin(['UPH', 'LPH','PH', 9999])]['Storeys']
# encoded_df

In [24]:
#lower ph floor ratio check index 1928
floor_index = encoded_df.columns.get_loc("floor")
encoded_df.iloc[[encoded_df[encoded_df['floor'].str.lower() == 'lph'].index], floor_index] = (encoded_df[encoded_df['floor'].str.lower() == 'lph']['Storeys'] - 1) / \
encoded_df[encoded_df['floor'].str.lower() == 'lph']['Storeys']

In [25]:
top_fl = encoded_df.loc[encoded_df['floor'].str.lower().isin(['ph', 'uph'])]['Storeys']
floor_ratio = top_fl.fillna(1.0) / top_fl.fillna(1.0)

In [26]:
# ph and uph calculation 
encoded_df.iloc[floor_ratio.index, floor_index] = floor_ratio.values

In [27]:
encoded_df.head(3)

,id,link,name,size_range,listing_price,price_per_ft2,shower,parking,dom,maint_fee,region,street,unit_num,latitude,longitude,taxes,description,pics,floorplans,suite_num,floor,room_num,Building_url,Address,Pre_constr,Corp#,Expected_Completion,Number_Units,Storeys,Smallest_Suite,Largest_Suite,Transit_score,Description,Maint_fee_covers,balcony_Enclosed,balcony_Juliette,balcony_None,balcony_Open,balcony_Terrace,balcony_Unknown,bed_1,bed_1+1,bed_2,bed_2+1,bed_3,bed_3+1,bed_4,bed_4+1,bed_Studio,bed_Studio & 1,locker_Common,locker_Ensuite,locker_Ensuite-Owned,locker_Exclusive,locker_No,locker_Owned,exposure_e,exposure_ew,exposure_n,exposure_ne,exposure_ns,exposure_nw,exposure_s,exposure_se,exposure_sw,exposure_w,Neighbourhood_Agincourt North,Neighbourhood_Agincourt South-Malvern West,Neighbourhood_Alderwood,Neighbourhood_Annex | U of T,Neighbourhood_Banbury | Don Mills,Neighbourhood_Bathurst Manor,Neighbourhood_Bay St. Corridor,Neighbourhood_Bayview Village,Neighbourhood_Bayview Woods-Steeles,Neighbourhood_Bedford Park-Nortown,Neighbourhood_Beechborough-Greenbrook,Neighbourhood_Bendale,Neighbourhood_Birchcliffe-Cliffside,Neighbourhood_Black Creek,Neighbourhood_Briar Hill-Belgravia,Neighbourhood_Bridle Path | Sunnybrook | York Mills,Neighbourhood_Broadview North,Neighbourhood_Brookhaven-Amesbury,Neighbourhood_Cabbagetown,Neighbourhood_Casa Loma,Neighbourhood_Church St. Corridor,Neighbourhood_CityPlace,Neighbourhood_Clairlea | Birchmount,Neighbourhood_Clanton Park,Neighbourhood_Cliffcrest,Neighbourhood_Corktown,Neighbourhood_Distillery District,Neighbourhood_Don Valley Village,Neighbourhood_Dorset Park,Neighbourhood_Dovercourt-Wallace Emerson-Junction,Neighbourhood_Downsview | Roding | CFB,Neighbourhood_East End-Danforth,Neighbourhood_Edenbridge | Humber Valley,Neighbourhood_Eglinton East,Neighbourhood_Englemount | Lawrence,Neighbourhood_Eringate | Centennial | West Deane,Neighbourhood_Etobicoke West Mall,Neighbourhood_Flemingdon Park,Neighbourhood_Forest Hill North,Neighbourhood_Forest Hill South,Neighbourhood_Fort York,Neighbourhood_Grange Park,Neighbourhood_Guildwood,Neighbourhood_Henry Farm,Neighbourhood_High Park North,Neighbourhood_High Park-Swansea,Neighbourhood_Hillcrest Village,Neighbourhood_Humber Heights | Westmount,Neighbourhood_Humber Summit,Neighbourhood_Humbermede,Neighbourhood_Humewood-Cedarvale,Neighbourhood_Islington | City Centre West,Neighbourhood_Junction Area,Neighbourhood_Keelesdale-Eglinton West,Neighbourhood_Kennedy Park,Neighbourhood_Kensington Market,Neighbourhood_King West,Neighbourhood_Kingsview Village | The Westway,Neighbourhood_L'Amoreaux,Neighbourhood_Lansing-Westgate,Neighbourhood_Lawrence Park North,Neighbourhood_Lawrence Park South,Neighbourhood_Leaside,Neighbourhood_Liberty Village,Neighbourhood_Little Italy,Neighbourhood_Little Portugal,Neighbourhood_Long Branch,Neighbourhood_Malvern,Neighbourhood_Markland Wood,Neighbourhood_Milliken,Neighbourhood_Mimico,Neighbourhood_Morningside,Neighbourhood_Moss Park,Neighbourhood_Mount Dennis,Neighbourhood_Mount Olive | Silverstone | Jamestown,Neighbourhood_Mount Pleasant East,Neighbourhood_Mount Pleasant West,Neighbourhood_Newtonbrook East,Neighbourhood_Newtonbrook West,Neighbourhood_O'Connor-Parkview,Neighbourhood_Oakridge,Neighbourhood_Parkwoods-Donalda,Neighbourhood_Pleasant View,Neighbourhood_Queen West,Neighbourhood_Regent Park,Neighbourhood_Rockcliffe-Smythe,Neighbourhood_Roncesvalles,Neighbourhood_Rosedale-Moore Park,Neighbourhood_Rouge E11,Neighbourhood_Scarborough Village,Neighbourhood_South Parkdale,Neighbourhood_South Riverdale | Leslieville,Neighbourhood_St. Andrew | Windfields,Neighbourhood_St. James Town,Neighbourhood_St. Lawrence,Neighbourhood_Steeles,Neighbourhood_Stonegate-Queensway,Neighbourhood_Tam O'Shanter | Sullivan,Neighbourhood_The Beaches,Neighbourhood_The Core,Neighbourhood_The Waterfront,Neighbourhood_Thorncliffe Park,Neighbourhood_Victoria Village,Neighbourhood_West Hill,Neighbourhood_West Humber | Clairville,Neighbourhood_Westminste

#### Encode ordered list of size_range

In [28]:
import collections
le_size = preprocessing.LabelEncoder()
le_size.fit(encoded_df['size_range'])
ordered_size_range = {}
for range_of_size in list(le_size.classes_):
    ordered_size_range[int(range_of_size.split('-')[0])] = range_of_size
ordered_size_range = collections.OrderedDict(sorted(ordered_size_range.items()))
# print (ordered_size_range)
ordered_range_list =[]
for _, ordered_range in ordered_size_range.items():
    ordered_range_list.append(ordered_range)

ordered_range_list.insert(0, '0')
ordered_range_list.insert(4, '700-799')

In [29]:
replace_map_comp = {'size_range' : {k: v for k,v in zip(ordered_range_list,list(range(1,len(ordered_range_list)+1)))}}

In [30]:
df_copy = encoded_df.copy()
df_copy.replace(replace_map_comp, inplace=True)

In [31]:
df_copy['listing_price_int'] = [int(x.strip('$').replace(",", "")) for x in df_copy['listing_price']]

In [32]:
df_copy = df_copy.drop('listing_price', axis = 1)

In [33]:
df_copy['price_per_ft2'] = df_copy['price_per_ft2'].str.replace(',', '')
df_copy['price_per_ft2'] = df_copy['price_per_ft2'].str.replace('$', '')
df_copy['price_per_ft2'] = df_copy['price_per_ft2'].replace({'N/A': np.nan})

In [34]:
df_copy = df_copy[df_copy['price_per_ft2'].notnull()]


In [35]:
df_copy['price_per_ft2'] = df_copy['price_per_ft2'].astype(float) 

In [36]:
df_copy['taxes_per_ft2'] = df_copy['taxes'] / (df_copy['listing_price_int']/df_copy['price_per_ft2'])
df_copy['maint_fee_per_ft2'] = df_copy['maint_fee'] / (df_copy['listing_price_int']/df_copy['price_per_ft2'])

In [37]:
# This is a list of the columns that are of interst to the model training

X_cols = ['size_range', 
       'shower', 'parking', 'dom','latitude', 'longitude',
#         'Storeys',
        'taxes_per_ft2',
        'Transit_score',
        'maint_fee_per_ft2',
        'balcony_Enclosed',
        'balcony_Juliette',
        'balcony_None',
        'balcony_Open',
        'balcony_Terrace',
        'balcony_Unknown',
        'bed_1',
        'bed_1+1',
        'bed_2',
        'bed_2+1',
        'bed_3',
        'bed_3+1',
        'bed_4',
        'bed_4+1',
        'bed_Studio',
        'bed_Studio & 1',
#         'bed_nan',
        'locker_Common',
        'locker_Ensuite',
        'locker_Ensuite-Owned',
        'locker_Exclusive',
        'locker_No',
        'locker_Owned',
        'exposure_e',
        'exposure_ew',
        'exposure_n',
        'exposure_ne',
        'exposure_ns',
        'exposure_nw',
        'exposure_s',
        'exposure_se',
        'exposure_sw',
        'exposure_w',
        'Neighbourhood_Agincourt North',
        'Neighbourhood_Agincourt South-Malvern West',
        'Neighbourhood_Alderwood',
        'Neighbourhood_Annex | U of T',
        'Neighbourhood_Banbury | Don Mills',
        'Neighbourhood_Bathurst Manor',
        'Neighbourhood_Bay St. Corridor',
        'Neighbourhood_Bayview Village',
        'Neighbourhood_Bayview Woods-Steeles',
        'Neighbourhood_Bedford Park-Nortown',
        'Neighbourhood_Beechborough-Greenbrook',
        'Neighbourhood_Bendale',
        'Neighbourhood_Birchcliffe-Cliffside',
        'Neighbourhood_Black Creek',
        'Neighbourhood_Briar Hill-Belgravia',
        'Neighbourhood_Bridle Path | Sunnybrook | York Mills',
        'Neighbourhood_Broadview North',
        'Neighbourhood_Brookhaven-Amesbury',
        'Neighbourhood_Cabbagetown',
        'Neighbourhood_Casa Loma',
        'Neighbourhood_Church St. Corridor',
        'Neighbourhood_CityPlace',
        'Neighbourhood_Clairlea | Birchmount',
        'Neighbourhood_Clanton Park',
        'Neighbourhood_Cliffcrest',
        'Neighbourhood_Corktown',
        'Neighbourhood_Distillery District',
        'Neighbourhood_Don Valley Village',
        'Neighbourhood_Dorset Park',
        'Neighbourhood_Dovercourt-Wallace Emerson-Junction',
        'Neighbourhood_Downsview | Roding | CFB',
        'Neighbourhood_East End-Danforth',
        'Neighbourhood_Edenbridge | Humber Valley',
        'Neighbourhood_Eglinton East',
        'Neighbourhood_Englemount | Lawrence',
        'Neighbourhood_Eringate | Centennial | West Deane',
        'Neighbourhood_Etobicoke West Mall',
        'Neighbourhood_Flemingdon Park',
        'Neighbourhood_Forest Hill North',
        'Neighbourhood_Forest Hill South',
        'Neighbourhood_Fort York',
        'Neighbourhood_Grange Park',
        'Neighbourhood_Guildwood',
        'Neighbourhood_Henry Farm',
        'Neighbourhood_High Park North',
        'Neighbourhood_High Park-Swansea',
        'Neighbourhood_Hillcrest Village',
        'Neighbourhood_Humber Heights | Westmount',
        'Neighbourhood_Humber Summit',
        'Neighbourhood_Humbermede',
        'Neighbourhood_Humewood-Cedarvale',
        'Neighbourhood_Islington | City Centre West',
        'Neighbourhood_Junction Area',
        'Neighbourhood_Keelesdale-Eglinton West',
        'Neighbourhood_Kennedy Park',
        'Neighbourhood_Kensington Market',
        'Neighbourhood_King West',
        'Neighbourhood_Kingsview Village | The Westway',
        'Neighbourhood_L\'Amoreaux',
        'Neighbourhood_Lansing-Westgate',
        'Neighbourhood_Lawrence Park North',
        'Neighbourhood_Lawrence Park South',
        'Neighbourhood_Leaside',
        'Neighbourhood_Liberty Village',
        'Neighbourhood_Little Italy',
        'Neighbourhood_Little Portugal',
        'Neighbourhood_Long Branch',
        'Neighbourhood_Malvern',
        'Neighbourhood_Markland Wood',
        'Neighbourhood_Milliken',
        'Neighbourhood_Mimico',
        'Neighbourhood_Morningside',
        'Neighbourhood_Moss Park',
        'Neighbourhood_Mount Dennis',
        'Neighbourhood_Mount Olive | Silverstone | Jamestown',
        'Neighbourhood_Mount Pleasant East',
        'Neighbourhood_Mount Pleasant West',
        'Neighbourhood_Newtonbrook East',
        'Neighbourhood_Newtonbrook West',
        'Neighbourhood_O\'Connor-Parkview',
        'Neighbourhood_Oakridge',
        'Neighbourhood_Parkwoods-Donalda',
        'Neighbourhood_Pleasant View',
        'Neighbourhood_Queen West',
        'Neighbourhood_Regent Park',
        'Neighbourhood_Rockcliffe-Smythe',
        'Neighbourhood_Roncesvalles',
        'Neighbourhood_Rosedale-Moore Park',
        'Neighbourhood_Rouge E11',
        'Neighbourhood_Scarborough Village',
        'Neighbourhood_South Parkdale',
        'Neighbourhood_South Riverdale | Leslieville',
        'Neighbourhood_St. Andrew | Windfields',
        'Neighbourhood_St. James Town',
        'Neighbourhood_St. Lawrence',
        'Neighbourhood_Steeles',
        'Neighbourhood_Stonegate-Queensway',
        'Neighbourhood_Tam O\'Shanter | Sullivan',
        'Neighbourhood_The Beaches',
        'Neighbourhood_The Core',
        'Neighbourhood_The Waterfront',
        'Neighbourhood_Thorncliffe Park',
        'Neighbourhood_Victoria Village',
        'Neighbourhood_West Hill',
        'Neighbourhood_West Humber | Clairville',
        'Neighbourhood_Westminster | Branson',
        'Neighbourhood_Weston',
        'Neighbourhood_Wexford-Maryvale',
        'Neighbourhood_Willowdale East',
        'Neighbourhood_Willowdale West',
        'Neighbourhood_Willowridge | Martingrove | Richview',
        'Neighbourhood_Woburn',
        'Neighbourhood_Woodbine Corridor',
        'Neighbourhood_Yonge and Bloor',
        'Neighbourhood_Yonge-Eglinton',
        'Neighbourhood_Yonge-St. Clair',
        'Neighbourhood_York University Heights',
        'Neighbourhood_Yorkdale | Glen Park',
        'Neighbourhood_Yorkville',
        'Developer_1001 Roselawn Avenue Inc',
        'Developer_109 Front Street East Inc.',
        'Developer_123 Eginton Avenue East Inc',
        'Developer_1410839 Ontario Limited',
        'Developer_2 Ridelle Avenue Co-Ownership Inc',
        'Developer_5 St.Joseph Development Ltd',
        'Developer_779857 Ontario Inc.',
        'Developer_Aberfoyle Developments Inc.',
        'Developer_All King Investment Inc',
        'Developer_Alterra & Finner Space Corporation',
        'Developer_Alterra Group',
        'Developer_Amexon Development Corporationn',
        'Developer_Aragon Properties Ltd.',
        'Developer_Arsando Investments Ltd',
        'Developer_Aspen Ridge Homes',
        'Developer_Atrium on Queens Quay Inc.',
        'Developer_B.G. Schickedanz Homes Inc.',
        'Developer_BLVD Developments',
        'Developer_BSaR Group of Companies',
        'Developer_Baghai Development Limited',
        'Developer_Banton Investments Limited',
        'Developer_Barclay-Grayson Group',
        'Developer_Barney River Investments',
        'Developer_Bayview Green Developments Inc.',
        'Developer_Bazis Inc. and Plaza',
        'Developer_Bazis Inc./Metropia/Plaza',
        'Developer_Bazis International',
        'Developer_Beaverbrook Homes',
        'Developer_Beaverhall Homes and Graywood Developments',
        'Developer_Belmont Construction',
        'Developer_Bentley Estates Limited',
        'Developer_Beverhall Homes and Graywood Developments Ltd',
        'Developer_Bomin Construction Ltd',
        'Developer_Bradsil Ltd',
        'Developer_Bramalea Limited',
        'Developer_Bramalea Ltd',
        'Developer_Brandy Lane Homes',
        'Developer_Bransfield Construction Ltd',
        'Developer_Burnac Corporation',
        'Developer_Cadillac Fairview Corporation Limited',
        'Developer_Cam-Nest Developments Limited',
        'Developer_Campeau Corporation',
        'Developer_Camrost Felcorp',
        'Developer_Camrost York Development Corporation',
        'Developer_Camrost York Developments',
        'Developer_Camrost- Felcorp Inc',
        'Developer_Camrost-Felcorp',
        'Developer_CanAlfa',
        'Developer_CanAlfa Group',
        'Developer_Canderel Stoneridge',
        'Developer_Capital Developments',
        'Developer_Castle Group',
        'Developer_Cenpac & Toddglen',
        'Developer_Cenpac Developments',
        'Developer_Centre Town Developments Limited',
        'Developer_CentreCourt Developments Inc.',
        'Developer_Centrecourt',
        'Developer_Century Plaza of Toronto Inc',
        'Developer_Chelsea Gate Inc',
        'Developer_Chesnut Homes',
        'Developer_Chestnut Park Residences Inc',
        'Developer_CityCore Developments & INVAR Building Corp',
        'Developer_Cityscape',
        'Developer_Cityscape Niagara Corp.',
        'Developer_Cityzen Development Corporation',
        'Developer_Cityzen Development Corporation & Myriad Group',
        'Developer_Concord Adex',
        'Developer_Concord Adex Development Corp',
        'Developer_Concord Adex Developments',
        'Developer_Concord Adex Developments Corp.',
        'Developer_Concorde',
        'Developer_Concorde Square Limited',
        'Developer_Conservatory Group',
        'Developer_Context Development',
        'Developer_Context Developments',
        'Developer_Cresford',
        'Developer_Cresford Developments',
        'Developer_Cressford Developments',
        'Developer_Crestford',
        'Developer_Crossroads Apartments Limited',
        'Developer_Curated Properties',
        'Developer_Daniels',
        'Developer_Daniels Corp',
        'Developer_Daniels Corporation',
        'Developer_Davies Smith Developments',
        'Developer_Deltan Realty Ltd',
        'Developer_Devron Developments',
        'Developer_Diamante Development Corporation',
        'Developer_Diamante Developments',
        'Developer_Diamondcorp',
        'Developer_Dogleg Developments Ltd',
        'Developer_Dream Maker Developments Inc',
        'Developer_Dundee Kilmer Developments',
        'Developer_Dundee Kilmer Develpments',
        'Developer_Dundee Realty Corporation',
        'Developer_Dynasty Court Properties Ltd.',
        'Developer_East of Bay Developments Corporation',
        'Developer_Eatons Group of Hotels',
        'Developer_Edilcan',
        'Developer_Edilcan Development',
        'Developer_Elad Canada',
        'Developer_Elm Developments',
        'Developer_Empire',
        'Developer_Empire Communities',
        'Developer_Emporis',
        'Developer_Enderby Developments Limited',
        'Developer_Estobrad Development Corporation',
        'Developer_Evergreen Villa Homes',
        'Developer_FNB Investments Ltd',
        'Developer_Fernbrook Homes',
        'Developer_Fernbrook Homes, Menkes, Cityzen Developments',
        'Developer_Firm Capital Properties Inc',
        'Developer_First Avenue Properties',
        'Developer_First College Grenville Holdings Inc',
        'Developer_Five Kenneth Avenue Ltd',
        'Developer_Forest Green Homes',
        'Developer_Four Wind Fbm Ltd.',
        'Developer_Freed Developments',
        'Developer_Freg Developments Ltd',
        'Developer_Gairloch',
        'Developer_Garden Terrace Developments Inc',
        'Developer_Gemterra Development',
        'Developer_George Wimpey Canada Limited',
        'Developer_George Wimpey Canada Ltd',
        'Developer_Georgian Group',
        'Developer_Ghods Builders Inc',
        'Developer_Ghods Builders Inc.',
        'Developer_Golraf Investments Ltd',
        'Developer_Graydon Hall Estates Limited',
        'Developer_Great Gulf',
        'Developer_Great Gulf Homes',
        'Developer_Great Gulf Homes & Menkes Developments',
        'Developer_Great Gulf Ltd',
        'Developer_Great Land (Kenneth) Residences Inc',
        'Developer_Great Lands Corporation',
        'Developer_Green Park',
        'Developer_Greenland Group',
        'Developer_Greenpark Homes',
        'Developer_Greenrock Alexus Limited',
        'Developer_Greenwin Construction',
        'Developer_Greenwin Ltd',
        'Developer_Greystone Walk Limited',
        'Developer_Greywood Developments',
        'Developer_Grossman Holding Limited',
        'Developer_Grozbord, King & Associates Limited',
        'Developer_H&R; Developments',
        'Developer_H&R; Developments, Lanterra Developments, Lifetime Developments',
        'Developer_Hanson Construction Inc',
        'Developer_Harhay Construction Management Ltd',
        'Developer_Harmony Developments II Inc.',
        'Developer_Harmony Park Properties Inc.',
        'Developer_Harry Stinson',
        'Developer_Hasam Investments Limited',
        'Developer_Heathcliffe Developments Limited',
        'Developer_Helmsbridge Holdings Limited',
        'Developer_Henderson Development',
        'Developer_Huang & Danczkay Ltd',
        'Developer_Huang & Dansky',
        'Developer_Huang and Dansky',
        'Developer_Icon Homes',
        'Developer_Identity Developments Incorporated',
        'Developer_Intracorp',
        'Developer_Intraurban Projects',
        'Developer_Invar Building Corp',
        'Developer_JSC Investments Ltd',
        'Developer_Jason H. Fane',
        'Developer_Jubilation Holdings Inc',
        'Developer_Kantium Developments',
        'Developer_King\'s Terrace on the Humber Limited',
        'Developer_Knightstone Capital Management Inc.',
        'Developer_Kolter Group',
        'Developer_Korank Properties Limited',
        'Developer_Kuhlmax Developments Ltd',
        'Developer_L & A Development (Canada) Ltd',
        'Developer_Lakeview Forest Development Ltd',
        'Developer_Lamb Development Corp',
        'Developer_Lamb Development Corp.',
        'Developer_Landmark & Urbancorp',
        'Developer_Lanet Building Corporation',
        'Developer_Lansino Holdings Limited',
        'Developer_Lanterra',
        'Developer_Lanterra & HR Developments & Malibu Investments',
        'Developer_Lanterra Developemnts, Lifetime Developments',
        'Developer_Lanterra Developments',
        'Developer_Lanterra Developments/ H&R; Developments/ Lifetime Developments',
        'Developer_Lash Developments',
        'Developer_Lawrence Avenue Group Limited',
        'Developer_Leaside Eglinton Holdings Inc.',
        'Developer_Lee Corporate Center Inc',
        'Developer_Lee Corporate Centre Inc',
        'Developer_Lee Development Group',
        'Developer_Liberty Development',
        'Developer_Lifetime Developments',
        'Developer_Lomax',
        'Developer_MOD Developments',
        'Developer_Madison Group',
        'Developer_Madison Homes',
        'Developer_Malibu Investments',
        'Developer_Malibu Investments, The Alterra Group, Urban Capital Property Group, Alit Canada, West Development Group',
        'Developer_Marchant Building Corporation',
        'Developer_Marr Contractors Ltd',
        'Developer_Mastercraft Group',
        'Developer_Mattamy Homes',
        'Developer_May Tower Developments LTD',
        'Developer_Mazel Construction Ltd',
        'Developer_Menkes',
        'Developer_Menkes Development',
        'Developer_Menkes Development Inc',
        'Developer_Menkes Developments',
        'Developer_Menkes Developments Inc',
        'Developer_Menkes Developments Limited',
        'Developer_Merton Development Corporation',
        'Developer_Minto',
        'Developer_Minto Developments',
        'Developer_Minto Group',
        'Developer_Minto Group Inc',
        'Developer_Minto Group Inc.',
        'Developer_Minto Longbranch',
        'Developer_Mizrahi Khalili Developments',
        'Developer_Moksem Corporation',
        'Developer_Mollenhauer Construction',
        'Developer_Monarch',
        'Developer_Monarch Construction',
        'Developer_Monarch Construction Limited',
        'Developer_Monarch Development Corporation',
        'Developer_Monarch Developments',
        'Developer_Monarch Developments Ltd',
        'Developer_Monarch Group',
        'Developer_Monarch Waterview Development Limited',
        'Developer_Montevallo Developments Limited',
        'Developer_Morguard Residential',
        'Developer_Neighbourhood Development Limited',
        'Developer_Neudorfer Corporation',
        'Developer_Norstar Developments',
        'Developer_Norstar Group',
        'Developer_Ogee Holdings Ltd',
        'Developer_Olympia and York Developments',
        'Developer_Olympia and York Developments Ltd',
        'Developer_Onni Group',
        'Developer_Overon Investments Limited',
        'Developer_Parkview-Kingsdale Holdings Limted',
        'Developer_Peel-Elder Developments Limited',
        'Developer_Pemberton Group',
        'Developer_Pemberton Group Inc',
        'Developer_Phantom Developments',
        'Developer_Philmor Group',
        'Developer_Pianosi Development Corp',
        'Developer_Pinnacle International',
        'Developer_Pinnacle International Properties Ltd',
        'Developer_Pivot Investment Limited',
        'Developer_Plaza',
        'Developer_Plaza Corp',
        'Developer_Plus Development Group Corp',
        'Developer_Pointe of View Developments Inc.',
        'Developer_Promary Properties Inc.',
        'Developer_QuadCam Development Group',
        'Developer_Queen Scrop(Dwight)Inc',
        'Developer_RMIC Building Corporation',
        'Developer_Reemark Sterling II Limited',
        'Developer_Remington Group',
        'Developer_Remmington Group',
        'Developer_River Dell Holdings Ltd',
        'Developer_Rocklyn Holding Limited',
        'Developer_Rockport Group',
        'Developer_Roselawn Properties',
        'Developer_Rowdenmoor Investments Limited',
        'Developer_Royal Grand Developments',
        'Developer_Royal Park Homes',
        'Developer_Royal-Dundas Developments Limited',
        'Developer_Rylar Development Ltd',
        'Developer_Rylar Developments',
        'Developer_SRM Development LLC',
        'Developer_Shane Baghai',
        'Developer_Sheppard Point Holding LTD.',
        'Developer_Sherwood Green Homes Limited',
        'Developer_Shiu Pong',
        'Developer_Sorbara Group',
        'Developer_Starwood Group',
        'Developer_State Building Group',
        'Developer_Steeles Hilda Developments Ltd',
        'Developer_Steven & Webster Callahan',
        'Developer_Strasscorp Holdings Inc',
        'Developer_Streetcar Developments',
        'Developer_Sundial Homes',
        'Developer_Sunset Homes',
        'Developer_TWS Developments Inc.',
        'Developer_Tas Design Build',
        'Developer_Tas Designbuild',
        'Developer_The Brown Group',
        'Developer_The Conservatory Group',
        'Developer_The Daniels Group',
        'Developer_The Fountains of Edenbridge Inc.',
        'Developer_The Karleton Company Ltd',
        'Developer_The Pemberton Group',
        'Developer_The Riedel Group',
        'Developer_The Sher Corporation',
        'Developer_The Torchin Group',
        'Developer_The Tradewinds Developments Inc',
        'Developer_Thorncroft Estates Ltd',
        'Developer_Times Group Corp',
        'Developer_Times Group Corporation',
        'Developer_Toddgeln Construction Ltd',
        'Developer_Toddglen Construction, Creson Investments',
        'Developer_Tor-Bel Group',
        'Developer_Torbel Group',
        'Developer_Torbel Group & Urban Fabric Development',
        'Developer_Tower Hill Development Corp.',
        'Developer_Tre Memovia Inc',
        'Developer_Triangle West Developments',
        'Developer_Tribute Communities',
        'Developer_Tridel',
        'Developer_Tridel Corporation',
        'Developer_Triloft Developments',
        'Developer_Triumph Developments',
        'Developer_Tussel Corporation Limited',
        'Developer_United Lands Corporation',
        'Developer_United Lands Corporation Ltd.',
        'Developer_Unknown',
        'Developer_Urban Capital Property Group',
        'Developer_Urban Corp',
        'Developer_Urbancorp',
        'Developer_Urbancorp Development',
        'Developer_V.H.L. Developments Inc.',
        'Developer_V.H.L. Developments Ltd.',
        'Developer_Valentine Developments Limited',
        'Developer_Valfour Developments',
        'Developer_Vandyk',
        'Developer_Victoria & Lawrence (Yorktown) Ltd',
        'Developer_Victoria and Grey Company',
        'Developer_Vienkuhl Construction Ltd',
        'Developer_W.B. Sullivan Construction Limited',
        'Developer_WB Sullivan Construction Ltd',
        'Developer_West Hill Developments',
        'Developer_West Hill Redevelopment Company Limited',
        'Developer_West Hill Redevelopment Company Ltd',
        'Developer_Westclair Developments Inc',
        'Developer_Woodside Pointe Developments',
        'Developer_Worsley Urban Partners',
        'Developer_York Square Developments Inc',
        'Property_Mgmt_Co_360 Community Management',
        'Property_Mgmt_Co_360 Community Management Limited',
        'Property_Mgmt_Co_AA Property Management',
        'Property_Mgmt_Co_APM',
        'Property_Mgmt_Co_Andrejs Management',
        'Property_Mgmt_Co_Atrens Management Group',
        'Property_Mgmt_Co_Aykler Management',
        'Property_Mgmt_Co_Balance Property Management',
        'Property_Mgmt_Co_Berkley Property Management',
        'Property_Mgmt_Co_Briarlane Property Management',
        'Property_Mgmt_Co_Brilliant Property Management',
        'Property_Mgmt_Co_Brookfield Condominium Services',
        'Property_Mgmt_Co_CIE Property Management',
        'Property_Mgmt_Co_Canlight Hall Management',
        'Property_Mgmt_Co_Cape Property Management',
        'Property_Mgmt_Co_Castle Condo Management',
        'Property_Mgmt_Co_City Sites Property Management',
        'Property_Mgmt_Co_Clp Property Management',
        'Property_Mgmt_Co_Comfield Property Management',
        'Property_Mgmt_Co_Comfort Property Management',
        'Property_Mgmt_Co_Condominium Living Management',
        'Property_Mgmt_Co_Connium Management',
        'Property_Mgmt_Co_Cpo/Canadian Properties Operator Management Inc.',
        'Property_Mgmt_Co_Crespro Management',
        'Property_Mgmt_Co_Crossbridge Condominium Services',
        'Property_Mgmt_Co_Daniels Home Management',
        'Property_Mgmt_Co_Daniels Property Management',
        'Property_Mgmt_Co_Del Property Management',
        'Property_Mgmt_Co_Dove Square Management',
        'Property_Mgmt_Co_Dream Maker Property Management',
        'Property_Mgmt_Co_Duka Property Management',
        'Property_Mgmt_Co_Eastway Property Management',
        'Property_Mgmt_Co_Eco Condo Management',
        'Property_Mgmt_Co_Elite Property Management',
        'Property_Mgmt_Co_Equity Ici',
        'Property_Mgmt_Co_First Service Property Management',
        'Property_Mgmt_Co_FirstService Residential',
        'Property_Mgmt_Co_FirstService Residential (Formerly Crosby Property Management)',
        'Property_Mgmt_Co_Gatsby Management',
        'Property_Mgmt_Co_General Property Management',
        'Property_Mgmt_Co_Goldview Property Management',
        'Property_Mgmt_Co_Gpm Property Management',
        'Property_Mgmt_Co_Grenadier Property Management',
        'Property_Mgmt_Co_Gsa Property Management',
        'Property_Mgmt_Co_Harmony Property Management',
        'Property_Mgmt_Co_Horizon Property Management',
        'Property_Mgmt_Co_Icc Property Management',
        'Property_Mgmt_Co_Icon Condomimiums',
        'Property_Mgmt_Co_Icon Property Management',
        'Property_Mgmt_Co_Jtb Management Group Inc',
        'Property_Mgmt_Co_Kindle Property Management',
        'Property_Mgmt_Co_Kipling Realty Management',
        'Property_Mgmt_Co_L & H Property Management',
        'Property_Mgmt_Co_Larlyn Property Management',
        'Property_Mgmt_Co_Lexington Property Management',
        'Property_Mgmt_Co_Living Properties',
        'Property_Mgmt_Co_MF Properties Ltd',
        'Property_Mgmt_Co_Malvern Condominium Property Management',
        'Property_Mgmt_Co_Malvern Property Management',
        'Property_Mgmt_Co_Manor Crest Management Ltd',
        'Property_Mgmt_Co_Manorcrest Management',
        'Property_Mgmt_Co_Maple Ridge Community Management',
        'Property_Mgmt_Co_Maple Ridge Property Management',
        'Property_Mgmt_Co_Mareka Property Management',
        'Property_Mgmt_Co_Menkes Property Management',
        'Property_Mgmt_Co_Menres Property Management',
        'Property_Mgmt_Co_Meritus Group Management Inc',
        'Property_Mgmt_Co_Nadlan-Harris Property Management',
        'Property_Mgmt_Co_Nalan Property Management',
        'Property_Mgmt_Co_Networks Property Management',
        'Property_Mgmt_Co_New City Property Management',
        'Property_Mgmt_Co_Newton-Trelawney',
        'Property_Mgmt_Co_Newton-Trewlany Property Management',
        'Property_Mgmt_Co_Northcan Property Management',
        'Property_Mgmt_Co_Nouvelle Management',
        'Property_Mgmt_Co_Online Property Management',
        'Property_Mgmt_Co_Orion Management',
        'Property_Mgmt_Co_PRO-HOUSE MANAGEMENT LIMITED',
        'Property_Mgmt_Co_Pal/Max Property Management',
        'Property_Mgmt_Co_Percel Professional Management',
        'Property_Mgmt_Co_Percel Property Management',
        'Property_Mgmt_Co_Philmore Group Ltd',
        'Property_Mgmt_Co_Principle Property Management',
        'Property_Mgmt_Co_Pro-House Management Ltd',
        'Property_Mgmt_Co_Prohouse Management',
        'Property_Mgmt_Co_Quadlib',
        'Property_Mgmt_Co_Quadlib Developments',
        'Property_Mgmt_Co_RBA Property Management',
        'Property_Mgmt_Co_Remington Facilities Management',
        'Property_Mgmt_Co_Royal Grand Property Management',
        'Property_Mgmt_Co_Royale Grande Property Management',
        'Property_Mgmt_Co_Self Managed',
        'Property_Mgmt_Co_Shelter Canadian Properties',
        'Property_Mgmt_Co_Shelter Property Management',
        'Property_Mgmt_Co_Shiu Pong Management',
        'Property_Mgmt_Co_Shui Pong Management',
        'Property_Mgmt_Co_Simerra',
        'Property_Mgmt_Co_Simsons Management',
        'Property_Mgmt_Co_Skywater Property Management',
        'Property_Mgmt_Co_St George Property Management',
        'Property_Mgmt_Co_Sterling Property Management',
        'Property_Mgmt_Co_Strategic Property Management',
        'Property_Mgmt_Co_Summerhill Property Management',
        'Property_Mgmt_Co_Taft Forward Property Management',
        'Property_Mgmt_Co_Taylor Property Management',
        'Property_Mgmt_Co_Terrawood Property Management',
        'Property_Mgmt_Co_Times Property Management',
        'Property_Mgmt_Co_Tony Kung & Associates',
        'Property_Mgmt_Co_Tse Management',
        'Property_Mgmt_Co_Unknown',
        'Property_Mgmt_Co_Vero Property Management',
        'Property_Mgmt_Co_Whitehill Residential',
        'Property_Mgmt_Co_Wilson Blanchard',
        'Property_Mgmt_Co_Woroz Management Inc',
        'Property_Mgmt_Co_YL Hendler Ltd.',
        'Property_Mgmt_Co_Zoran Properties',
        'floor_ratio']

In [38]:
df_copy_no_null = df_copy.dropna(subset = ['Storeys', 'Transit_score', 'floor_ratio'])
# df_copy_no_null

In [39]:
X = df_copy_no_null[X_cols].values
y = df_copy_no_null['price_per_ft2'].values

In [40]:
X_cols.append('price_per_ft2')
model_input = df_copy_no_null[X_cols]

In [41]:
corr_matrix = model_input.corr()
corr_matrix['price_per_ft2'].sort_values(ascending = False)

price_per_ft2                                         1.000000
Neighbourhood_Bay St. Corridor                        0.296270
bed_1                                                 0.254828
Developer_Lifetime Developments                       0.252501
taxes_per_ft2                                         0.245474
bed_1+1                                               0.217822
Neighbourhood_Yorkville                               0.216630
Neighbourhood_King West                               0.208583
bed_Studio                                            0.204802
Neighbourhood_Yonge and Bloor                         0.192285
maint_fee_per_ft2                                     0.182751
balcony_Open                                          0.177657
Property_Mgmt_Co_Kipling Realty Management            0.175698
Property_Mgmt_Co_FirstService Residential             0.172934
Property_Mgmt_Co_Icon Property Management             0.164964
locker_Owned                                          0

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [43]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()

### Start training

In [44]:
#XGBOOST
import xgboost
xgb = xgboost.XGBRegressor(n_estimators=200, learning_rate=0.09, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=6)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.09, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [45]:
y_pred_xgb = xgb.predict(X_test)

In [46]:
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(y_test, y_pred_xgb)))

84.0718903402534


In [47]:
# grid search
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.metrics import mean_squared_error, make_scorer
# parameters = {'n_estimators' : [150, 200, 250],
#               'learning_rate':[0.01, 0.1],
parameters = {'n_estimators' : [200],
              'min_child_weight': [1, 5, 10],
#               'gamma': [0.5, 1, 1.5, 2.],
#               'subsample': [0.6, 0.8, 1.0],
#               'colsample_bytree': [0.6, 0.8, 1.0],
#               'max_depth': [4, 5, 6, 7]
              }
def RMSE(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(abs(mse))
    print ('RMSE: %2.3f' % rmse)
    return rmse

def rmse_scorer():
    return make_scorer(RMSE, greater_is_better=False) 

xgb_cv = xgboost.XGBRegressor(objective='reg:linear')
search = GridSearchCV(xgb_cv, parameters, scoring = 'neg_mean_squared_error', cv = 3, verbose = 4)
search.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] min_child_weight=1, n_estimators=200 ............................
[CV]  min_child_weight=1, n_estimators=200, score=-6833.770481868986, total=   3.3s
[CV] min_child_weight=1, n_estimators=200 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV]  min_child_weight=1, n_estimators=200, score=-7561.638810572729, total=   2.3s
[CV] min_child_weight=1, n_estimators=200 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.6s remaining:    0.0s


[CV]  min_child_weight=1, n_estimators=200, score=-5787.958912035246, total=   2.3s
[CV] min_child_weight=5, n_estimators=200 ............................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.9s remaining:    0.0s


[CV]  min_child_weight=5, n_estimators=200, score=-6958.047824145156, total=   2.9s
[CV] min_child_weight=5, n_estimators=200 ............................
[CV]  min_child_weight=5, n_estimators=200, score=-8169.423415665757, total=   2.5s
[CV] min_child_weight=5, n_estimators=200 ............................
[CV]  min_child_weight=5, n_estimators=200, score=-6061.355126834189, total=   3.0s
[CV] min_child_weight=10, n_estimators=200 ...........................
[CV]  min_child_weight=10, n_estimators=200, score=-7469.844493102249, total=   2.7s
[CV] min_child_weight=10, n_estimators=200 ...........................
[CV]  min_child_weight=10, n_estimators=200, score=-7896.373496048161, total=   2.5s
[CV] min_child_weight=10, n_estimators=200 ...........................
[CV]  min_child_weight=10, n_estimators=200, score=-6064.847554525002, total=   2.2s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   23.8s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [200], 'min_child_weight': [1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=4)

In [48]:
search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [49]:
search.cv_results_

{'mean_fit_time': array([2.62192218, 2.7866931 , 2.43806473]),
 'std_fit_time': array([0.43929327, 0.21479444, 0.19560244]),
 'mean_score_time': array([0.00639343, 0.0071524 , 0.00644167]),
 'std_score_time': array([0.0011038 , 0.00107251, 0.00083043]),
 'param_min_child_weight': masked_array(data=[1, 5, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[200, 200, 200],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_child_weight': 1, 'n_estimators': 200},
  {'min_child_weight': 5, 'n_estimators': 200},
  {'min_child_weight': 10, 'n_estimators': 200}],
 'split0_test_score': array([-6833.77048187, -6958.04782415, -7469.8444931 ]),
 'split1_test_score': array([-7561.63881057, -8169.42341567, -7896.37349605]),
 'split2_test_score': array([-5787.95891204, -6061.35512683, -6064.84755453]),
 'mean_test_score': array([-6727.78940149, -7062.9